In [3]:
import pandas as pd
from sklearn.manifold import TSNE
from pathlib import Path

In [12]:
path = Path("../public/data/games.dr.json")
df = pd.read_json(path)
df["cluster"] = df["cluster"].apply(lambda x: max(x, key=lambda y: y["value"])["name"] if len(x) > 0 else "none")
tagNames = {}

# collect tags names
for _, d in df.iterrows():
    for t in d["tags"]:
        tagNames[t["id"]] = t["name"]

tagList = [tid for tid in tagNames.keys()]

# get values
tags = [["false" for _ in range(df.shape[0])] for _ in tagList]

for i, d in df.iterrows():
    for t in d["tags"]:
        j = tagList.index(t["id"])
        tags[j][i] = "true"

df.drop(columns=["tags"], inplace=True)
idx = 0
for tid in tagList:
    df[tagNames[tid]] = tags[idx]
    idx += 1

df.to_csv(path.with_suffix(".csv"), index=False)
df

C:\Users\beckerfa\AppData\Local\Temp\ipykernel_2148\1970468511.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tagNames[tid]] = tags[idx]
C:\Users\beckerfa\AppData\Local\Temp\ipykernel_2148\1970468511.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tagNames[tid]] = tags[idx]
C:\Users\beckerfa\AppData\Local\Temp\ipykernel_2148\1970468511.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colu

,x,y,id,name,cluster,chunk: continuous,com: npc and player,exp: homogeneous,ginfo: exploration,ginfo: remember self,...,top: art,set-g: steampunk,set-s: underwater,mech: physical save points,set-s: africa,top: parenthood,top: sports,mini: strategy,set-t: antiquity,com: free gestures
0,-14.325387,41.791396,1,Her Story,hotbar,true,true,true,true,true,...,false,false,false,false,false,false,false,false,false,false
1,0.683069,41.682644,2,L.A. Noire,none,false,false,true,true,false,...,false,false,false,false,false,false,false,false,false,false
2,12.782077,37.092761,3,The Wolf Among Us,none,false,false,true,true,false,...,false,false,false,false,false,false,false,false,false,false
3,8.432699,29.110300,4,Sherlock Holmes: Crimes & Punishments,none,false,false,true,true,false,...,false,false,false,false,false,false,false,false,false,false
4,-3.265335,33.115906,5,Sherlock Holmes: Consulting Detective,none,false,false,true,true,true,...,false,false,false,false,false,false,false,false,false,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,0.513898,-34.331832,414,Patrician III,none,true,false,false,false,true,...,false,false,false,false,false,false,false,true,false,false
384,-2.733511,-37.290272,415,Pharaoh™: A New Era,none,false,false,true,false,false,...,false,false,false,false,true,false,false,false,true,false
385,-0.184625,-36.037412,416,Farming Simulator 25,command queue,true,false,true,false,true,...,false,false,false,false,false,false,false,true,false,false
386,-26.430475,21.944461,417,5D Chess With Multiverse Time Travel,none,true,false,true,false,false,...,false,false,false,false,false,false,false,false,false,false


In [4]:
def project(df):
    tsne = TSNE(n_components=2, perplexity=float(max(10.0, min(50.0, df.shape[1] / 5))))
    result = tsne.fit_transform(df.to_numpy())
    return result

def process(df):
    ndf = df.copy()
    cols = df.dtypes[df.dtypes == "object"].index.values
    for c in cols:
        vals = df[c].unique().tolist()
        ndf[c] = df[c].apply(lambda x: vals.index(x))

    return ndf.fillna(0)

def transform_all(names):
    for n in names:
        file = Path(f"../public/data/{n}.csv")
        df = pd.read_csv(file)
        ndf = process(df)
        print(ndf)
        coords = project(ndf)
        df["x"] = coords[:,0]
        df["y"] = coords[:,1]
        df.to_csv(file.with_suffix(".dr.csv"), index=False)

In [5]:
# files = ["iris", "heart_disease_uci", "cereal", "anxiety_depression"]
files = ["ibm-employee-attrition-test"]
transform_all(files)

       Employee ID  Age  Gender  Years at Company  Job Role  Monthly Income  \
0            52685   36       0                13         0            8029   
1            30585   35       0                 7         1            4563   
2            54656   50       0                 7         1            5583   
3            33442   58       0                44         2            5525   
4            15667   39       0                24         1            4604   
...            ...  ...     ...               ...       ...             ...   
14895        16243   56       1                42         0            7830   
14896        47175   30       1                15         1            3856   
14897        12409   52       0                 5         1            5654   
14898         9554   18       0                 4         1            5276   
14899        73042   59       1                48         1            3774   

       Work-Life Balance  Job Satisfaction  Perform

In [38]:
file = Path("../public/data/anxiety_depression.csv")
df = pd.read_csv(file)
df

,Age,Gender,Education_Level,Employment_Status,Sleep_Hours,Physical_Activity_Hrs,Social_Support_Score,Anxiety_Score,Depression_Score,Stress_Level,...,Chronic_Illnesses,Medication_Use,Therapy,Meditation,Substance_Use,Financial_Stress,Work_Stress,Self_Esteem_Score,Life_Satisfaction_Score,Loneliness_Score
0,56,Male,Bachelor's,Unemployed,6.0,0.4,3,4,2,9,...,0,None,0,1,None,4,3,7,5,1
1,69,Female,Bachelor's,Retired,8.8,2.8,6,18,7,6,...,0,None,1,0,None,1,4,7,4,6
2,46,Female,Master's,Employed,5.3,1.6,5,5,13,8,...,0,None,0,1,None,8,7,8,1,1
3,32,Female,High School,Unemployed,8.8,0.5,4,6,3,4,...,1,None,0,0,None,7,4,8,4,4
4,60,Female,Bachelor's,Retired,7.2,0.7,2,7,15,3,...,0,None,1,1,Frequent,8,9,5,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,23,Female,Other,Student,5.1,0.1,7,16,15,5,...,0,Occasional,0,0,None,2,8,7,6,5
1196,43,Female,Master's,Employed,8.9,0.4,9,19,4,9,...,0,None,0,1,None,8,9,7,6,4
1197,71,Female,PhD,Retired,6.0,0.1,2,15,4,2,...,0,None,0,1,None,2,9,9,3,7
1198,21,Male,Other,Employed,8.5,2.7,8,12,20,2,...,1,None,0,0,None,8,5,1,3,7


In [ ]:


ndf = process(df)
ndf

,Age,Gender,Education_Level,Employment_Status,Sleep_Hours,Physical_Activity_Hrs,Social_Support_Score,Anxiety_Score,Depression_Score,Stress_Level,...,Chronic_Illnesses,Medication_Use,Therapy,Meditation,Substance_Use,Financial_Stress,Work_Stress,Self_Esteem_Score,Life_Satisfaction_Score,Loneliness_Score
0,56,0,0,0,6.0,0.4,3,4,2,9,...,0,0,0,1,0,4,3,7,5,1
1,69,1,0,1,8.8,2.8,6,18,7,6,...,0,0,1,0,0,1,4,7,4,6
2,46,1,1,2,5.3,1.6,5,5,13,8,...,0,0,0,1,0,8,7,8,1,1
3,32,1,2,0,8.8,0.5,4,6,3,4,...,1,0,0,0,0,7,4,8,4,4
4,60,1,0,1,7.2,0.7,2,7,15,3,...,0,0,1,1,1,8,9,5,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,23,1,3,3,5.1,0.1,7,16,15,5,...,0,1,0,0,0,2,8,7,6,5
1196,43,1,1,2,8.9,0.4,9,19,4,9,...,0,0,0,1,0,8,9,7,6,4
1197,71,1,4,1,6.0,0.1,2,15,4,2,...,0,0,0,1,0,2,9,9,3,7
1198,21,0,3,2,8.5,2.7,8,12,20,2,...,1,0,0,0,0,8,5,1,3,7
